In [ ]:
import pandas as pd
import numpy as np
import datetime
from sodapy import Socrata
import s3fs
import os
print(os.listdir("/home/jovyan/work"))

In [ ]:
# Need to install packages till docker image is standard jupyter/datascience-notebook from dockerhub 
#!pip install sodapy
#!pip install s3fs

In [ ]:
client = Socrata("data.cityofnewyork.us", None)
results = client.get("j25v-6efb", limit=20000)
df = pd.DataFrame.from_records(results)
df[:5]

In [ ]:
df.info()

In [ ]:
df = df[['hour_beginning', 'pedestrians', 'towards_manhattan', 'towards_brooklyn', 'weather_summary', 'temperature', 'precipitation', 'events']]

In [ ]:
df = df.sort_values(by='hour_beginning').reset_index()
df.drop(labels=['index'], axis=1, inplace=True)
df[:5]

In [ ]:
df['events'].fillna(0, inplace=True)

In [ ]:
df.fillna(method='ffill', inplace=True)

In [ ]:
df['hour_beginning'] = pd.to_datetime(df['hour_beginning'])
df['hour_beginning'][:5]

In [ ]:
# Write file with data frame directly to S3
# key and secret possesion is needed
bytes_to_write = df.to_csv(None).encode()
fs = s3fs.S3FileSystem(key='FakeKey', secret='FakeSecretKey')
with fs.open('s3://brooklyn-bridge-pedestrians/brooklyn_bridge_data.csv', 'wb') as f:
    f.write(bytes_to_write)